<a href="https://colab.research.google.com/github/rcafoley/PotvinFuglevand2017MUFatigueModel/blob/main/BaseMUmodelPotvinFuglevand2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Potvin & Fuglevand 2017 Phenomenological 120 Motor Unit Fatigue Model converted to python from Matlab

In [24]:
# %% Import Packages

import math
import numpy as np
import pandas as pd

In [25]:
# %% Setup timing functions

#create our timing functions, more tic and toc variables can be added to time 
#multiple sectionss
import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [26]:
tic()

# %% Model Static Physiological Input Parameters

nu = 120           # number of neurons (ie. motor units) in the modeled pool 

samprate = 10      # sample rate (10 Hz is suggested)
res = 100          # resolution of activations (set = 10 for 0.1 activation resolution, 100 for 0.01)
hop = 20           # allows for hopping through activations to more rapidly find that which meets the threshold (10 means every 1/10th of maxact)
r = 50             # range of recruitment thresholds (30 or 50)

fat = 180          # range of fatigue rates across the motor units (300 best)
rec = 180          # range of recovery rates across the motor units (300 best)
FatFac = 0.0225    # fatigue factor (FF/S) percent of peak force of MU per second 

tau = 22           # 22 based on Revill & Fuglevand (2011)
adaptSF = 0.67     # 0.67 from Revill & Fuglevand (2011)
ctSF = 0.379       # 0.379 based on Shields et al (1997)

mthr = 1           # minimum recruitment threshold
a0 = 1              # recruitment gain paramter (1)
minfr = 8          # minimum firing rate (8)
recminfr = 8       # firing rate where recovery element is engaged
pfr1 = 35          # peak firing rate of first recruited motor unit (35)
pfrL = 25          # peak firing rate of last recruited motor unit (25)
mir = 1            # slope of the firing rate increase vs excitation (1)
rp = 100           # range of twitch tensions (100)
rt = 3             # range of contraction times (3 fold)
tL = 90            # longest contraction time (90) 

In [27]:
#Create isotonic data -----------------------------------

fthscale = 0.9             # sets %MVC level for the trial duration (100% MVC is 1.00)
con = '0.90'               # for output file names
fthtime = 100              # duration to run trial (seconds)

fthsamp = fthtime * samprate 
fth = np.array(range(fthsamp+1))
fth = np.select([fth < fthsamp], [fthscale], fth)

In [28]:
# %% MU Pool Physiological Calculations from the Fuglevand, Winter & Patla (1993) Model 

    #array of samples for fth
ns = np.zeros(fthsamp+1)
ns = np.array(range(1,(fthsamp+1)))

In [29]:
#Recruitment Threshold Excitation (thr) 
thr = np.zeros(nu+1)
n = np.array(range(0, nu+1))     #had to add a 0 to the start of n, so any array-wise math with n will have to insert a 0 at the start, do the math then remove the zero
b0 = math.log(r + (1-mthr)) / (nu-1)             # this was modified from Fuglevand et al (1993) RTE(i) equation (1)
thr = list(range(nu+1))
for p in range(1,nu+1):                                 # as that did not create the exact range of RTEs (ie. 'r') entered
    thr[p] = a0 * math.exp((p-1) * b0) - (1 - mthr)


In [30]:
# Peak Firing Rate (frp) 
# modified from Fuglevand et al (1993) PFR equation (5) to remove thr(1) before ratio
frdiff = pfr1-pfrL
thr = np.array(thr)
# thrTemp = np.insert(thr, 0, 1)
frp = pfr1 - frdiff*((thr[n] - thr[1])/(r - thr[1]))
# frp = np.delete(frpTemp, 0)

maxex = thr[nu] + (pfrL - minfr)/mir       # maximum excitation
maxact = round(maxex * res)                # max excitation x resolution
ulr = 100 * thr[nu]/maxex                  # recruitment threshold (%) of last motor unit
# thr = np.delete(thrTemp, 0)

In [31]:
# Firing Rates for each MU with increased excitation (act)
mufr = np.zeros((nu+1, (maxact+1)))
for mu1 in range(1,nu+1):
    for act in range(1,maxact+1):
        acti = (act)/res
        if acti >= thr[mu1]:
            mufr[mu1, act] = mir * (acti - thr[mu1]) + minfr  
        if mufr[mu1, act] > frp[mu1]:   
            mufr[mu1, act] = frp[mu1]
        elif acti < thr[mu1]:
            mufr[mu1, act] = 0
k = np.array(range(maxact+1))  #range of excitation levels

In [32]:
# Twitch peak force (P) 
b1 = math.log(rp)/(nu-1)                 # this was modified from Fuglevand et al (1993) P(i) equation (13)
Ptemp1 = map(lambda n: math.exp(b1 * (n-1)), n)     # this is changed for python to be able to apply arraywise calculations easier to a single output
Ptemp2 = list(Ptemp1)               #convert to list to array and removes 0 point to get indexing back on track 
Ptemp3 = np.array(Ptemp2)
# P = np.delete(Ptemp3, 0)
P = Ptemp3
P[0] = 0 

In [33]:
# Twitch contraction time (ct)        
c1 = math.log(rp)/math.log(rt)                # scale factor
ct = np.zeros(nu+1)        
for mu2 in range(1,nu+1):                       # assigns contraction times to each motor unit (moved into loop)
    ct[mu2] = tL * (1/P[mu2])**(1/c1)

In [34]:
# Normalized motor unit firing rates (nmufr) with increased excitation (act)
nmufr = np.zeros((nu+1, maxact+1))
for mu3 in range(1,nu+1):                       
    for act in range(1,maxact+1):
        nmufr[mu3, act] = ct[mu3] * (mufr[mu3, act] / 1000)   # same as CT / ISI  

In [35]:
# Motor unit force, relative to full fusion (Pr) with increasing excitation
# based on Figure 2 of Fuglevand et al (1993)
sPr = 1 - math.exp(-2 * (0.4**3))
Pr = np.zeros((nu+1, maxact+1))
for mu4 in range(1,nu+1):                       
    for act in range(1,maxact+1):
        if nmufr[mu4,act] <= 0.4:                      #linear portion of curve
            Pr[mu4,act] = nmufr[mu4,act]/0.4 * sPr   #Pr = MU force relative to rest 100% max excitation of 67
        if nmufr[mu4,act] > 0.4:
            Pr[mu4,act] = 1 - math.exp(-2 * (nmufr[mu4,act]**3))

In [36]:
# Motor unit force (muP) with increased excitation
muP = np.zeros((nu+1, maxact+1))
for mu5 in range(1,nu+1):
    for act in range(1,maxact+1): 
        muP[mu5,act] = Pr[mu5,act] * P[mu5]

totalP = np.sum(muP,0)                                # sum of forces across MUs for each excitation (dim 1)
maxP = totalP[maxact]

In [37]:
# Total Force across all motor units when rested
Pnow = np.zeros((nu+1, fthsamp+2))
Pnow[:,1] = P[:]

In [38]:
# %% Calculation of Fatigue Parameters (recovery currently set to zero in this version)
    
    # note, if rp = 100 & fat = 180, there will be a 100 x 180 = 1800-fold difference in
    # the absolute fatigue of the highest threshold vs the lowest threshold.
    # The highest threshold MU will only achieve ~57% of its maximum (at 25 Hz), so the actual range of fatigue
    # rates is 1800 x 0.57 = 1026 
    

# fatigue rate for each motor unit  (note: "log" means "ln" in Matlab)
b2 = math.log(fat)/(nu-1)       
mufatrateTemp1 = map(lambda n: math.exp(b2 * (n-1)), n)      
mufatrateTemp2 = list(mufatrateTemp1)               #convert to list to array and removes 0 point to get indexing back on track 
mufatrateTemp3 = np.array(mufatrateTemp2)
# mufatrate = np.delete(mufatrateTemp3, 0)
mufatrate = mufatrateTemp3 
mufatrate[0] = 0


fatigue = np.zeros(nu+1)
for mu6 in range(1,nu+1):
    fatigue[mu6] = mufatrate[mu6] * (FatFac / fat) * P[mu6]    
        # the full fatigue rate is mufatrate[mu] * [FatFac / fat] * Pr[mu,act] * P[mu]
        # the only variable is the relative force: Pr([u,act], so this part is calculated once here

In [39]:
# Establishing the rested excitation required for each target load level (if 0.1% resolution, then 0.1% to 100%)    
startactTemp = np.zeros(101)    
for force in range(101):
    startactTemp[force] = 0    # excitation will never be lower than that needed at rest for a given force
    for act1 in range(1,maxact+1):      # so it speeds the search up by starting at this excitation
        if (totalP[act1]/maxP * 100) < force:
            startactTemp[force] = act1-1 
# startact = np.delete(startactTemp, 0)
startact = startactTemp 


Pchangecurves = np.zeros((nu+1, maxact))
for act in range(maxact):
    for mu8 in range(1,nu+1):
        Pchangecurves[mu8,act] = fatigue[mu8] * Pr[mu8, act] * P[mu8]  # just used for graphical display

In [40]:
mes = 'start of fatigue analysis'
print(mes)

# %% Moving through force time-history and determing the excitation required to meet the target force at each time

TmuPinstant = np.zeros((nu+1, fthsamp+1)) 
m = np.zeros(fthsamp+1)
mufrFAT = np.zeros((nu+1, fthsamp+1))
ctFAT = np.zeros((nu+1, fthsamp+1))
ctREL = np.zeros((nu+1, fthsamp+1))
nmufrFAT = np.zeros((nu+1, fthsamp+1))
PrFAT = np.zeros((nu+1, fthsamp+1))
muPt = np.zeros((nu+1, fthsamp+1))
TPt = np.zeros(fthsamp+1)
Ptarget = np.zeros(fthsamp+1)
Tact = np.zeros(fthsamp+1)
Pchange = np.zeros((nu+1, fthsamp+1))
TPtMAX = np.zeros(fthsamp+1)
muPtMAX = np.zeros((nu+1, fthsamp+1)) 
muON = np.zeros(nu+1)
adaptFR = np.zeros((nu+1,fthsamp+1))
Rdur = np.zeros(nu+1)
Rdur2 = np.zeros((nu+1,fthsamp+1))
acttemp = np.zeros((fthsamp+1, maxact))
muPna = np.zeros((nu+1, fthsamp+1))
muForceCapacityRel = np.zeros((nu+1, fthsamp+1))
timer = 0

  
for i in range(1,fthsamp+1): 

    if (i) == (timer + 1) * samprate * 60:         # shows a timer value every 15 seconds
        timer = timer + 1
        current = (i) / samprate
        print(current)
    
        
    force = round(fth[i] * 100) + 1            # used to start at the minimum possible excitation (lowest it can be is 1)
    if force > 100:                                # so start with excitation needed for fth(i) when rested (won't be lower than this)
        force = 100


    s = startact[force] - (5 * res)            # starts a little below the minimum    
    if s < 1:
        s = 1


    acthop = round((maxact / hop))              # resets 'acthop' to larger value for new sample
    act = int(s)                                     # start at lowest value then start jumping by 'acthop'
    for a1 in range(1,maxact+1):                        # this starts at the mimimum (s) then searches for excitation required to meet the target
        acttemp[i,a1] = act
        for mu9 in range(1,nu+1): 
            # MU firing rate adaptation modified from Revill & Fuglevand (2011)
            # this was modified to directly calculate the firing rate adaption, as 1 unit change in excitation causes 1 unit change in firing rate
            # scaled to the mu threshold (highest adaptation for hightest threshold mu)

            if muON[mu9] > 0:
                Rdur[mu9] = (i - muON[mu9]+1)/samprate                             # duration since mu was recruited at muON
                Rdur2[mu9,i]  = Rdur[mu9]
            if Rdur[mu9] < 0:     
                Rdur[mu9] = 0 
                Rdur2[mu9,i]  = Rdur            
            
            adaptFR[mu9,i] = (thr[mu9]-1)/(thr[nu]-1) * adaptSF * (mufr[mu9,act] - minfr + 2) * (1 - math.exp(-1 * Rdur[mu9] / tau)) 
            if adaptFR[mu9,i] < 0:                                                # firing rate adaptation
                adaptFR[mu9,i] = 0

            mufrFAT[mu9,i] = mufr[mu9,act] - adaptFR[mu9,i]                           # adapted motor unit firing rate: based on time since recruitment
            mufrMAX = mufr[mu9,maxact] - adaptFR[mu9,i]                        # adapted FR at max excitation

            ctFAT[mu9,i] = ct[mu9] * (1 + ctSF * (1 - Pnow[mu9,i]/P[mu9]))             # corrected contraction time: based on MU fatigue
            ctREL[mu9,i] = ctFAT[mu9,i]/ct[mu9]

            nmufrFAT[mu9,i] = ctFAT[mu9,i] * (mufrFAT[mu9,i] / 1000)                 # adapted normalized Stimulus Rate (CT * FR)
            nmufrMAX = ctFAT[mu9,i] * (mufrMAX / 1000)                        # normalized FR at max excitation
        
            if nmufrFAT[mu9,i] <= 0.4:                                                 # fusion level at adapted firing rate 
                PrFAT[mu9,i] = nmufrFAT[mu9,i] / 0.4 * sPr                         #   linear portion of curve

            if nmufrFAT[mu9,i] > 0.4:                                                 #   non-linear portion of curve
                PrFAT[mu9,i] = 1 - math.exp(-2 * (nmufrFAT[mu9,i]**3))

            muPt[mu9,i] = PrFAT[mu9,i] * Pnow[mu9,i]                               # MU force at the current time (muPt): based on adapted postion on fusion curve
                
            if nmufrMAX <= 0.4:                                                   # fusion force at 100% maximum excitation
                PrMAX = nmufrMAX / 0.4 * sPr                                  

            if nmufrMAX > 0.4:
                PrMAX = 1 - math.exp(-2 * (nmufrMAX**3))

            muPtMAX[mu9,i] = PrMAX * Pnow[mu9,i]                               # Max MU force capacity at the current time 
   
    
        muPtTemp = sum(muPt,0)
        TPt[i] = muPtTemp[i]/maxP                                               # total sum of MU forces at the current time (TPt)
        TPtMAX[i] = sum(muPtMAX[:,i])/maxP
        
        # used to speed up the search for the right excitation to meet the current target
        if ((TPt[i] < fth[i]) and (act == maxact)):
            break

        if ((TPt[i]) < (fth[i])):
            act = act + acthop
            if act > (maxact):
                act = maxact

        if ((TPt[i] >= fth[i]) and (acthop == 1)):     
            break                                   # stop searching as the correct excitation is found
     
        if ((TPt[i] >= fth[i]) and (acthop > 1)):
            act = act - (acthop - 1)                # if the last large jump was too much, it goes back and starts increasing by 1
            if act < 1:
                act = 1

            acthop = 1

        # next excitation (act)
        
    for mu10 in range(1,nu+1):
        if ((muON[mu10] == 0) and ((act/res) >= thr[mu10])):         # can be modified to reset if the MU turns off
            muON[mu10] = i                                        # time of onset of mu recruitment (s)


    Ptarget[i] = TPt[i]        # modeled force level ?? do I need to do this, or can I just use TPt(i)
    Tact[i] = act              # descending (not adapted) excitation required to meet the target force at the current time
    
    # Calculating the fatigue (force loss) for each motor unit

    for mu11 in range(1,nu+1):
        if mufrFAT[mu11,i] >= 0:                                                 # Force loss of each MU for each interval
            Pchange[mu11,i] = -1 * (fatigue[mu11] / samprate) * PrFAT[mu11, i]       # based on % of MU fusion force
        #elif mufrFAT[mu11, i] < recminfr:                                        
             #Pchange[mu11,i] = recovery[mu11] / samprate   


        if i < 2:
            Pnow[mu11, i+1] = P[mu11]
               # Pnow[mu11, i+1] = 0  # Use this to start the muscle fully exhausted
        elif i >= 2:
            Pnow[mu11, i+1] = Pnow[mu11, i] + Pchange[mu11,i]                            # instantaneous strength of MU
                                                                                    # right now without adaptation
     
        if Pnow[mu11, i+1] >= P[mu11]:
            Pnow[mu11, i+1] = P[mu11]                                                   # does not let it increase past rested strength
    
    
        if Pnow[mu11, i+1] < 0:
            Pnow[mu11, i+1] = 0                                                       # does not let it fatigue below zero
    
# next motor unit
  
# next fthsamp


start of fatigue analysis
60.0


In [41]:
# %% Strength Calculations

Tstrength = np.zeros(fthsamp+1)  
for j in range(1,fthsamp+1):
    for mu in range(1,nu+1):
        muPna[mu,j] = Pnow[mu,j] * muP[mu,maxact] / P[mu]                          # non-adapted MU max force at 100% excitation (muPna)            

    Tstrength[j] = sum(muPna[:,j]) / maxP                                          # Current total strength without adaptation relative to max rested capacity

for g in range(1,fthsamp+1):                                   
    endurtime = g / samprate 
    if TPtMAX[g] < fth[g]:
        break            

In [42]:
# %% Output
                
EndStrength = (TPtMAX[fthsamp] * 100)

print(endurtime)
print(EndStrength)
            
for mu12 in range(1,nu+1):
    if mu12 == 0:
        mu12 = 1

    muForceCapacityRel[mu12,ns] = Pnow[mu12,ns]*100/P[mu12]  # for outputs below
    

sampleRate1 = ns/samprate
times = np.insert(sampleRate1,0,0)
forceTimeHistory = fth
activationLevel = Tact[:]/res/maxex * 100
strengthCapacity = Tstrength[:] * 100
targetForce = Ptarget[:] * 100
instantaneousStrength = TPtMAX[:]* 100


combo = pd.DataFrame(data = [times,forceTimeHistory,
                             activationLevel,strengthCapacity, targetForce, instantaneousStrength])
combo_t = combo.T
combo_t.columns = ["time","percentActivation","forceTimeHistory","totalStrengthCapacity","targetForce","maxInstantaneousForce"]


combo_t.to_csv(con + " A - Target - Act - Strength (no adapt) - Force - Strength (w adapt).csv") 
np.savetxt(con + " B - Firing Rate.csv", np.transpose(mufrFAT), delimiter=",")
np.savetxt(con + " C - Individual MU Force Time-History.csv", np.transpose(muPt), delimiter=",")
np.savetxt(con + " D - MU Capacity - relative.csv", np.transpose(muForceCapacityRel), delimiter=",")

#end of script 
toc() 

6.5
37.41500309741526
Elapsed time: 55.050385 seconds.

